In [1]:
!nvidia-smi

Thu May  9 10:56:00 2024       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.40.06              Driver Version: 551.23         CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce RTX 3070 ...    On  |   00000000:01:00.0  On |                  N/A |
| N/A   47C    P8             14W /  115W |    7842MiB /   8192MiB |      4%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [2]:
!curl http://0.0.0.0:8000/v1/models

{"object":"list","data":[{"id":"TheBloke/Mistral-7B-Instruct-v0.2-AWQ","object":"model","created":1715248640,"owned_by":"vllm","root":"TheBloke/Mistral-7B-Instruct-v0.2-AWQ","parent":null,"permission":[{"id":"modelperm-094c24ef7c734e77ae3d155af7783c2a","object":"model_permission","created":1715248640,"allow_create_engine":false,"allow_sampling":true,"allow_logprobs":true,"allow_search_indices":false,"allow_view":true,"allow_fine_tuning":false,"organization":"*","group":null,"is_blocking":false}]}]}

In [13]:
from openai import OpenAI
import concurrent.futures
import time

client = OpenAI(base_url="http://0.0.0.0:8000/v1", api_key="sk-xxx") # dummy key


def get_response(name, max_tokens = 100, prompt = "Write a story about llamas.", system_prompt = "You are a story writing assistant."):
    t0 = time.time()
    response = client.chat.completions.create(
        max_tokens=max_tokens,
        temperature=0.1,
        model="TheBloke/Mistral-7B-Instruct-v0.2-AWQ",
        messages=[
            #{"role": "system", "content": system_prompt}, MISTRAL does not use a system prompt
            {
                "role": "user",
                "content": system_prompt+" "+prompt
            }
        ])
    return name, response, time.time() - t0

In [14]:
name, response, t = get_response(
    0, max_tokens=250, 
    system_prompt='You are expert code assistant. Please reply to user queries in an accurate, clear and concise manner.',
    prompt = 'Tell me how I can download and run llama from huggingface.')
print(f'Took {t}s')
print(response.choices[0].message.content)

Took 7.250380277633667s
 To download and run Llama, a large language model from Hugging Face, you can follow these steps:

1. First, make sure you have Python installed on your system. You can download it from the official website: https://www.python.org/downloads/

2. Next, install the Transformers library, which is a Hugging Face library for state-of-the-art Natural Language Processing. You can install it using pip:

```
pip install transformers
```

3. Once the installation is complete, you can download the Llama model using the following command:

```
wget https://huggingface.co/LLM-Math/Llama-7B-hf/resolve/main/config.json
wget https://huggingface.co/LLM-Math/Llama-7B-hf/resolve/main/tokenizer_config.json
wget https://huggingface.co/LLM-Math/Llama-7B-hf/resolve/main/pytorch_model.bin
wget https://h


In [15]:
t0 = time.time()
threads = []
with concurrent.futures.ThreadPoolExecutor(max_workers=10) as executor:
    # Submit the tasks to the executor
    for i in range(10):
        threads.append(executor.submit(get_response, i, 250))
        print('Submited thread', i)

    # print results as and when they come in
    for task in concurrent.futures.as_completed(threads):
        name, response, exec_time = task.result()
        message = response.choices[0].message.content
        print(f'Request {name}, time: {exec_time:.2f}, n_tokens = {response.usage.total_tokens}')
        #print(message)

print('Total Execution time:', time.time()-t0)

Submited thread 0
Submited thread 1
Submited thread 2
Submited thread 3
Submited thread 4
Submited thread 5
Submited thread 6
Submited thread 7
Submited thread 8
Submited thread 9
Request 3, time: 9.10, n_tokens = 273
Request 9, time: 9.08, n_tokens = 273
Request 0, time: 9.12, n_tokens = 273
Request 4, time: 9.10, n_tokens = 273
Request 7, time: 9.09, n_tokens = 273
Request 6, time: 9.10, n_tokens = 273
Request 8, time: 9.09, n_tokens = 273
Request 1, time: 9.12, n_tokens = 273
Request 2, time: 9.11, n_tokens = 273
Request 5, time: 9.10, n_tokens = 273
Total Execution time: 9.125575065612793
